In [1]:
%set_env SPARK_HOME=/opt/spark
import os
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """\
--master spark://10.62.42.237:7077 
--packages datastax:spark-cassandra-connector:2.3.1-s_2.11,anguenot:pyspark-cassandra:0.9.0
--driver-memory 12g
--executor-memory 8g
--conf spark.local.dir=/var/spark
--conf spark.network.timeout=100000s
--conf spark.executor.heartbeatInterval=10000s
--conf spark.shuffle.registration.timeout=10000s
pyspark-shell
"""

import pyspark
from pyspark import SparkContext, SparkConf
findspark.init()
sc = pyspark.SparkContext(appName="BestRetry")

env: SPARK_HOME=/opt/spark


In [2]:
sc._conf.set("spark.sql.execution.arrow.enabled", "true")
sc._conf.set('spark.driver.maxResultSize', '10g')
sc._conf.set('spark.driver.cores', '4')
sc._conf.set('spark.executor.cores', '4')
# sc._conf.set('spark.driver.memory', '24g')
# sc._conf.set('spark.executor.memory', '24g')
sc._conf.set("spark.cores.max", 8)
sc._conf.set("spark.local.dir", "/var/spark")

# sc._conf.set("spark.network.timeout", "1000000s")
# sc._conf.set("spark.executor.heartbeatInterval", "1000000s")

# sc._conf.set("spark.cassandra.input.split.size_in_mb", 32)
# sc._conf.set("spark.cassandra.splitCount", 16)
# sc._conf.set("spark.cassandra.connection.host", "10.224.12.32")

sc._conf.set("spark.cassandra.connection.host", "10.62.1.118")
sc._conf.set("spark.cassandra.auth.username", "mlprw")
sc._conf.set("spark.cassandra.auth.password", "q4RgwD$wK7*z")

sc._conf.getAll()

[('spark.master', 'spark://10.62.42.237:7077'),
 ('spark.driver.cores', '4'),
 ('spark.deploy.spreadOut', 'true'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.driver.memory', '12g'),
 ('spark.local.dir', '/var/spark'),
 ('spark.files',
  'file:///root/.ivy2/jars/datastax_spark-cassandra-connector-2.3.1-s_2.11.jar,file:///root/.ivy2/jars/anguenot_pyspark-cassandra-0.9.0.jar,file:///root/.ivy2/jars/com.twitter_jsr166e-1.1.0.jar,file:///root/.ivy2/jars/org.joda_joda-convert-1.2.jar,file:///root/.ivy2/jars/io.netty_netty-all-4.0.33.Final.jar,file:///root/.ivy2/jars/commons-beanutils_commons-beanutils-1.9.3.jar,file:///root/.ivy2/jars/joda-time_joda-time-2.3.jar,file:///root/.ivy2/jars/org.scala-lang_scala-reflect-2.11.8.jar,file:///root/.ivy2/jars/commons-collections_commons-collections-3.2.2.jar,file:///root/.ivy2/jars/com.datastax.spark_spark-cassandra-connector_2.11-2.3.0.jar,file:///root/.ivy2/jars/net.razorvine_pyrolite-4.10.jar,file:///root/.ivy2/jars/net.razorvine_serpe

In [3]:
import pyspark
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.\
    master("spark://10.62.42.237:7077").\
    appName("BestRetry").\
    config(conf=sc._conf).\
    getOrCreate()
spark

sqlContext = SQLContext(sc)
sqlContext

In [4]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'


def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
            yield date

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
#     group['failed_transaction_hour'] = ''
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
#     group['failed_response_group'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        df_last_fail = group[ (group['renew_att_num'] == group['renew_att_num'].max()) & (group['cid'] != None) ]
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
#     last_attempt_date = last['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
#     last['failed_transaction_hour'] = df_failed_txn['transaction_hour']
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
#     last['failed_response_group'] =   df_failed_txn['response_message'].apply(group_response_msg)
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

    
udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

# df = spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-03-%' ")
df = spark.sql("SELECT * FROM cpg_transaction_history")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                            
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

def _map_to_pandas(rdds):
    """ Needs to be here due to pickling issues """
    return [pd.DataFrame(list(rdds))]

def toPandas(df, n_partitions=None):
    """
    Returns the contents of `df` as a local `pandas.DataFrame` in a speedy fashion. The DataFrame is
    repartitioned if `n_partitions` is passed.
    :param df:              pyspark.sql.DataFrame
    :param n_partitions:    int or None
    :return:                pandas.DataFrame
    """
    if n_partitions is not None: df = df.repartition(n_partitions)
    df_pand = df.rdd.mapPartitions(_map_to_pandas).collect()
    df_pand = pd.concat(df_pand)
    df_pand.columns = df.columns
    return df_pand

sdf_filtered = sdf\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")  #.toPandas()

pdf_all = toPandas(sdf_filtered, 4)  
    
print("# total time:", time.time() - time_start)      # total time: 2364.714904308319
    
# pdf_all.shape

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

In [8]:
#sdf_filtered.to_csv('best_retry_01_02.csv')
WORK_DIR = '/var/spark/ml_files/'
pdf_all.to_csv(WORK_DIR + 'best_retry_03.csv')
pdf_all.shape

(920431, 53)

In [7]:
pdf_all

,received_date,site_id,subscription_id,transaction_id,transaction_date,added_expiry_years,bank_code,bank_name,billing_country,bin,...,transaction_hour,transaction_type,success,failed_attempt_date,failed_day_of_month,failed_response_code,failed_response_message,failed_decline_type,days_between,failed_event_uuid
0,2018-03-27,avast,4679748701,6606831463,2018-03-27 00:47:41,,F26EBCDBE58D0EC4,USAA Federal Savings Bank,US,456367,...,00,Authorize,0,2018-03-12 00:45:30,3,502,304: Lost/Stolen Card,Hard Fraud,15,01806eb8-b6ab-4089-9cc6-d6a4685a3448
1,2018-03-30,avastbr,4781058901,6617138493,2018-03-30 00:48:50,,NON3DS,Banco do Brasil S.A.,BR,498401,...,00,Authorize,1,2018-03-25 00:54:16,3,530,7: Do Not Honor,do_not_honor,5,ee93c0aa-34e2-4585-b88b-3a7e3893ac7c
2,2018-03-28,adbehkr,11195528901,6611214263,2018-03-28 02:41:27,,NON3DS,"WOORICARD CO., LTD",KR,538920,...,02,Authorize,0,2018-03-14 01:58:52,3,27050,Do not try again/use alternate payment card.[6...,Hard Fraud,14,19f3d80e-844b-46f3-a13d-a8f0a0583beb
3,2018-03-29,adbehbr,10798133101,6614426433,2018-03-29 01:06:29,,862D58E247602019,ITAU UNIBANCO S.A.,BR,548474,...,01,Authorize,1,2018-03-15 01:33:58,3,530,7: Do Not Honor,do_not_honor,14,a0ca5920-f1f4-4270-8fee-2045b609f67d
4,2018-03-03,avast,4010512701,6528156723,2018-03-03 02:30:17,4,RB,La Banque Postale,FR,415059,...,02,Authorize,0,2018-03-03 02:30:19,3,27001,Transaction refused[05] [] [111800] [111800],txn_refused,0,d18b7672-4764-4ecb-bb18-7cc416b593bf
5,2018-03-30,avast,5343854301,6617090023,2018-03-30 00:43:15,3,NON3DS,BBVA BANCO FRANCES,AR,529329,...,00,Authorize,0,2018-03-30 00:43:16,3,27001,Transaction refused[05] [] [111800] [111800],txn_refused,0,54f62a99-7b88-4916-8c7a-c5a41102ef2f
6,2018-03-26,adbehap,9262717601,6604096753,2018-03-26 01:24:43,,NON3DS,SBI Cards and Payment Services Pte Ltd,IN,472642,...,01,Authorize,0,2018-03-05 01:36:21,3,27001,Transaction refused[05] [000000] [] [],txn_refused,21,a529594e-690a-43c8-9b6f-8e65a233c659
7,2018-03-15,adbehru,11121914701,6568069593,2018-03-15 02:31:39,,NON3DS,TINKOFF BANK,RU,553691,...,02,Authorize,1,2018-03-08 01:44:03,3,27001,Transaction refused[51] [000000] [] [],Insufficient Funds,7,70fd7b9b-5de8-4089-836e-493b5a38581d
8,2018-03-24,avast,9906987801,6597665833,2018-03-24 01:28:48,,E6B9CECFCAC4D317,JPMorgan Chase Bank N.A. - Debit,US,483314,...,01,Authorize,0,2018-03-09 01:26:40,3,530,51: DECLINED,declined,15,ce2b12b9-941d-45a8-8d58-495ba5ed57b0
9,2018-03-24,kasperus,9885148101,6597466793,2018-03-24 01:01:06,,D0ECAF1CD662089E,Equitable Bank,US,413976,...,01,Authorize,0,2018-03-07 09:05:32,3,530,51: DECLINED,declined,17,f4ddead6-68b9-45cd-9155-d5bf5528dea1


In [12]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

WORK_DIR = '/var/spark/ml_files/'
# start_date = '2018-04-01'
# end_date = '2018-06-30'

def convert_status(value):
    if value == 'Completed' or value == 'Reversed':
        return 1
    else:
        return 0
    
    
def cast_to_int(value, default=-1):
    try:
        return int(value)
    except (ValueError, TypeError):
        return default


#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))
            
def date_range_condition_values(start_date, end_date):
    dates = daterange(start_date, end_date)
    for idx, date in enumerate(dates):
        date_formatted = date.replace('-','')
        yield date_formatted

df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   



def merge_by_sub(group):

    
    #Assign default values to additonal columns
    group['failed_attempt_date'] = ''
    group['failed_day_of_month'] = 0
    group['failed_response_code'] = ''
    group['failed_response_message'] = ''
    group['failed_decline_type'] = ''
    group['days_between'] = 0
    group['failed_event_uuid'] = ''    
    
    
    
    last = pd.DataFrame() 
    df_success = group[group['success'] == 1]

    if not df_success.empty:
        df_last_attempt = df_success.iloc[0]
    else:
        df_last_fail = group[ (group['renew_att_num'] == group['renew_att_num'].max()) & (group['cid'] != None) ]
        if not df_last_fail.empty:
            df_last_attempt =  df_last_fail.iloc[0]
        else:
            group['event_uuid'] = '' 
            return group
    
    
    df_failed_txn = group[group['event_uuid'] != df_last_attempt['event_uuid']]
    if df_failed_txn.empty:
        group['event_uuid'] = '' 
        return group
    
    last = last.append( [df_last_attempt] * len(df_failed_txn),ignore_index=True ).set_index(df_failed_txn.index)
        
    failed_attempt_date = df_failed_txn['transaction_date_in_string']
    
    last['failed_attempt_date'] = failed_attempt_date
    last['failed_day_of_month'] = df_failed_txn['day_of_month']
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['failed_response_code'] = df_failed_txn['response_code']
    
    failed_response_msg = df_failed_txn['response_message']
    last['failed_response_message'] = failed_response_msg  
    
    last['failed_decline_type'] = df_failed_txn['response_message'].apply(decline_type) 
    last['days_between'] = last.apply(days_between_ds, axis=1)    
    last['failed_event_uuid'] =   df_failed_txn['event_uuid']   
        
    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)

    return abs((d2 - d1).days)

#Get difference between d2 and d1 in days.
def days_between_ds(df):
    d1 = to_date(df['failed_attempt_date'])
    d2 = to_date(df['transaction_date_in_string'])
    return abs((d2 - d1).days)

# @pandas_udf(sdf.schema, PandasUDFType.GROUPED_MAP)

        

# date_list = list(date_range_condition_values(start_date, end_date))


udfValueToStatus = udf(convert_status, IntegerType())  

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "MLP Cluster",
     pushdown "true")"""
spark.sql(createDDL)

df = spark.sql("SELECT * FROM cpg_transaction_history")

sdf =  df\
    .filter("!(renew_att_num == '1' and new_status in ('Completed','Reversed') and cid is null and status == 'New')")\
    .withColumn("success", udfValueToStatus("new_status"))\
    .withColumn("day_of_month", df["day_of_month"].cast(IntegerType()))\
    .withColumn("day_of_week", lower(col("day_of_week")))\
    .cache()

print(sdf.schema)
    
grouped_schema = sdf.schema.add("failed_attempt_date", StringType())\
                            .add("failed_day_of_month", IntegerType())\
                            .add("failed_response_code", StringType())\
                            .add("failed_response_message", StringType())\
                            .add("failed_decline_type", StringType())\
                            .add("days_between", IntegerType())\
                            .add("failed_event_uuid", StringType())
                    
                            
@pandas_udf(grouped_schema, PandasUDFType.GROUPED_MAP)
def merge_by_sub_spark(spark_df):
    pdf = pd.DataFrame(spark_df)
    result = pdf.groupby(['subscription_id', 'subsegment_id']).apply(merge_by_sub)
    return result                            
                            
time_start = time.time()

year=2018
months = ['04','05','06']
for m in months:
    yearmonth = str('%s-%s'%(year,m))
    condition = str("received_date like '%s%%'" % yearmonth)
    filename = str("retry_success_%s.csv" % yearmonth)
    sdf_by_month = sdf\
        .where(condition)\
        .filter(sdf.renew_att_num.isNotNull())\
        .groupby("subscription_id", "subsegment_id")\
        .apply(merge_by_sub_spark)\
        .filter("event_uuid != ''")
    
#     sdf_by_month.toPandas().to_csv(WORK_DIR + filename) 
    toPandas(sdf_by_month, 4).to_csv(WORK_DIR + filename) 
    
    
print("# total time:", time.time() - time_start)    
    
# pdf_all

StructType(List(StructField(received_date,DateType,true),StructField(site_id,StringType,true),StructField(subscription_id,StringType,true),StructField(transaction_id,StringType,true),StructField(transaction_date,TimestampType,true),StructField(added_expiry_years,StringType,true),StructField(bank_code,StringType,true),StructField(bank_name,StringType,true),StructField(billing_country,StringType,true),StructField(bin,StringType,true),StructField(card_brand,StringType,true),StructField(card_category,StringType,true),StructField(card_class,StringType,true),StructField(card_usage,StringType,true),StructField(cc_expiration_date,StringType,true),StructField(cid,StringType,true),StructField(date_increment,StringType,true),StructField(day_of_month,IntegerType,true),StructField(day_of_week,StringType,true),StructField(event_uuid,StringType,true),StructField(exp_setting_id,StringType,true),StructField(funding_source,StringType,true),StructField(issuer_country,StringType,true),StructField(merchant

ValueError: Length mismatch: Expected axis has 0 elements, new values have 53 elements

In [13]:
sc.stop()

In [23]:
import time

import pandas as pd


from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count, lit, lower
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.functions import udf
from pyspark.sql.types import *
import pyspark_cassandra
        
import calendar, datetime
from datetime import timedelta

# rdd = sc \
#     .cassandraTable("dev_sarsks", "cpg_transaction_history")\
#     .select('site_id')

# #     .map(lambda a:a)

# #     .where("received_date in ?", date_list)

# sdf = spark.read\
#     .format("org.apache.spark.sql.cassandra")\
#     .options(table="cpg_transaction_history", keyspace="dev_sarsks")\
#     .load() 

# df = sqlContext.createDataFrame(rdd, StructType([StructField('site_id',StringType(),True)]))

# df = sqlContext.read.format('org.apache.spark.sql.cassandra').options(table="cpg_transaction_history", keyspace="dev_sarsks")\
#     .where("received_date = ?", '2018-01-01')\
#     .load()
# df.select('id','date').where(df.date=='2017-12-01').show()
# df.where( df.received_date == '2018-01-01').head()
# df = sqlContext.createDataFrame(rdd, StructType([StructField('site_id',StringType(),True),StructField('payment_currency',StringType(),True)]))

# sdf = df.withColumn("received_date", df["received_date"].cast(StringType()))
# df = sqlContext.createDataFrame(rdd, sdf.schema)
# df.head()

createDDL = """CREATE OR REPLACE TEMPORARY VIEW cpg_transaction_history
     USING org.apache.spark.sql.cassandra
     OPTIONS (
     table "cpg_transaction_history",
     keyspace "dev_sarsks",
     cluster "Test Cluster",
     pushdown "true")"""
spark.sql(createDDL)

spark.sql("SELECT * FROM cpg_transaction_history WHERE received_date like '2018-03-%'").show()
# df.head()
# print(rdd.count)      

+-------------+-------+---------------+--------------+-------------------+------------------+----------------+--------------------+---------------+------+----------------+--------------------+----------+--------------------+------------------+----------------+--------------+------------+-----------+--------------------+--------------+--------------+--------------+-------------------+--------------------+-----------------+----------+-----------------+--------------+------------------+----------------+-----------------+------------------+-------------+--------------+------------------+----------------+--------------+-------------+--------------------+------+-------------+--------------------------+----------------+----------------+
|received_date|site_id|subscription_id|transaction_id|   transaction_date|added_expiry_years|       bank_code|           bank_name|billing_country|   bin|      card_brand|       card_category|card_class|          card_usage|cc_expiration_date|             cid|

In [9]:
# pdf_all['day_of_week'].value_counts(dropna=False)
# pdf_all.to_csv('/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-06-30.csv')
from IPython.display import display
pd.options.display.max_columns = None
display(pdf_all)

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success,first_attempt_date,first_day_of_month,first_transaction_hour,first_response_code,first_response_message,first_response_group,first_decline_type,days_between
0,20180418,avast,10001280501,673671cc-cc57-400c-b872-8ff9efb09a6d,None,NON3DS,Standard Chartered Bank Tanzania Limited,TZ,478680,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,0322,None,None,18,wednesday,None,Debit,TZ,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,DR_IRELAND-ENTITY,79.99,79.99,USD,Visa,netgiro-bms,3,79.99,79.99,USD,46971082700,27050,Do not try again/use alternate payment card.[0...,None,New,13258980000,,2018-04-18 06:21:29,2018-04-18 01:21:29,01,6681738323,Authorize,0,2018-04-03 01:07:59,3,01,27050,Do not try again/use alternate payment card.[0...,do_not_try,Hard Fraud,15
1,20180418,avastbr,10001441601,11cc8ab0-fe20-4e80-9d8c-2a7a856e7f17,None,NON3DS,ITAU UNIBANCO S.A.,BR,539083,MasterCard,(Unembossed) Standard MasterCard Card,Consumer,True credit (No PIN/Signature capability),1223,None,None,18,wednesday,None,Credit,BR,Avast Brazil,1883448942,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,99,29.018051,BRL,MasterCard,netgiro-br,3,99,29.018051,BRL,46969296500,27001,Transaction refused[62] [null] [] [],None,New,13259159000,,2018-04-18 06:02:11,2018-04-18 01:02:11,01,6681601083,Authorize,0,2018-04-08 01:03:36,8,01,27001,Transaction refused[62] [null] [] [],txn_refused,Hard Fraud,10
2,20180418,avast,10001635101,8bc95f07-77b3-4bd4-abfa-5fee6658eb63,None,57E041B0B2F3D9A5,Metabank,US,435880,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0625,None,None,18,wednesday,None,Prepaid,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,53.99,53.99,USD,Visa,firstdata,3,53.99,53.99,USD,46969386300,402,12: INV TRAN,None,New,13259490100,,2018-04-18 06:03:02,2018-04-18 01:03:02,01,6681607843,Authorize,0,2018-04-03 01:10:44,3,01,402,12: INV TRAN,Base,Base,15
3,20180425,defendbr,10001895901,9f5ecc0d-2529-4930-9fc5-93daf88c9e74,3,899EBF821D876833,ITAU UNIBANCO S.A.,BR,521397,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),0620,None,None,25,wednesday,None,Credit,BR,DRI*BitDefender,75FA81DE-5AC5-5DCF-E97C-8F80A08ADE1E,DR_BRAZIL-ENTITY,Declined,DR_BRAZIL-ENTITY,229.99,66.742732,BRL,MasterCard,pagador,3,229.99,66.742732,BRL,47202743400,530,7: Do Not Honor,None,New,13259627500,,2018-04-25 06:01:35,2018-04-25 01:01:35,01,6706709733,Authorize,0,2018-04-16 01:01:42,16,01,530,7: Do Not Honor,do_not_honor,do_not_honor,9
4,20180418,avast,10001966301,1bf185fb-88cd-457c-85a2-a697021fc538,None,0FDA420D7B7FDBDC,"Chase Bank USA, National Association",US,424631,Visa,Visa Business,Business,True credit (No PIN/Signature capability),0718,None,None,18,wednesday,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Declined,DR_INC-ENTITY,64.94,64.94,USD,Visa,firstdata,3,64.94,64.94,USD,46969539100,201,14: INV ACCT NUM,None,New,13259788300,,2018-04-18 06:05:07,2018-04-18 01:05:07,01,6681621783,Authorize,0,2018-04-03 01:12:21,3,01,201,14: INV ACCT NUM,Base,Base,15
5,20180519,adbehru,10002261401,0d1fbb95-019b-4f5f-8c53-f96ae78c2d0a,None,NON3DS,SBERBANK of Russia,RU,427609,Visa,Visa Classic,Consumer,NON USA Consumer Debit - No Pin Access,1018,RETRY_DECLINED.2,None,19,saturday,None,Debit,RU,DRI*Adobe Systems,1411163460-R